<a href="https://colab.research.google.com/github/rkap786/reading_idm/blob/main/BERTembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import transformers
import torch
import itertools
import numpy as np
from numpy import savetxt
from numpy import dot
from numpy.linalg import norm

#*** pretrained_model_vectorizer.py
# Library which includes all code that involves the use of pretrained
# models for contextualization
# Functions:
#    vectorize_with_pretrained_embeddings(sentences, labels): Given a list of text examples,
#    produces embeddings of dim 768 for each example
#***


In [2]:
from transformers import AutoTokenizer

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Tokenizer loaded successfully!


In [ ]:
# !pip install huggingface-cli
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import pandas as pd
f= pd.read_csv("/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Data/Processed/data_passage_ques_options_pv.csv")

In [ ]:
#f.head()
#f['Passage'][1]

'"No one but a man can do this," the business manager of the World, a New York newspaper, said to the young woman. The year was 1888. A popular book at the time told about a character who traveled around the world in 80 days. Now Nellie Bly, a young reporter for the newspaper, wanted to do it in real life.\n "Very well,\' said Nellie. "Start the man, and I\'ll start the same day for some other newspaper and beat him."\n In those days it was very unusual for a woman to travel alone. But Nellie Bly was an unusual woman. Her real name was Elizabeth Jane Cochran. When she was 20, she wrote a fiery letter to the editor of the Pittsburgh Dispatch. The editor was so impressed with her letter that he offered her a job as a writer.\n It wasn\'t considered "proper" to use a woman\'s name in a newspaper. So the editor signed Elizabeth\'s work Nellie Bly, a name from a popular song.\n Back then, women were only supposed to write about things considered to be "women\'s topics," such as fashion and 

In [5]:
# question= f['QuestionText']
# option1= f['option_correct_ans']
# option2= f['question_dis1']
# option3= f['question_dis2']
# option4= f['question_dis3']
# # Question + Correct answer
# ques_correct_answer= f['question_correct_ans']
# ques_distractor1= f['option_distractor1']
# ques_distractor2= f['option_distractor2']
# ques_distractor3= f['option_distractor3']


# Questions and all options - no passage
ques_alloptions_nopass = 'Question:' + f['QuestionText'] + '\n Correct answer: ' + f['option_correct_ans'] + "\n" + '\n Wrong answer 1: ' + f['option_distractor1'] + "\n" + '\n Wrong answer 2: ' + f['option_distractor2'] + "\n" + '\n Wrong answer 3: ' + f['option_distractor3']


# Passage +
ques_alloptions_tag = 'Question:' + f['QuestionText'] + '\n Correct answer: ' + f['option_correct_ans'] + "\n" + '\n Wrong answer 1: ' + f['option_distractor1'] + "\n" + '\n Wrong answer 2: ' + f['option_distractor2'] + "\n" + '\n Wrong answer 3: ' + f['option_distractor3'] + "\n" + f['Passage']




In [ ]:
# print(ques_distractor1_tag[1])
# print(ques_distractor1[1])
# print(f['QuestionText'][1])
# print("Everything in one \n" + ques_alloptions_tag[1])


 Read this sentence from paragraph 6.
 Traveling east across the Atlantic, Nellie took just one bag in order to move
 quickly. 
 How is this detail important to paragraph 1?
 Wrong answer 1: It shows how she is the same as the character in the book.
It shows how she is the same as the character in the book.
 Read this sentence from paragraph 6.
 Traveling east across the Atlantic, Nellie took just one bag in order to move
 quickly. 
 How is this detail important to paragraph 1?
Everything in one 
 Read this sentence from paragraph 6.
 Traveling east across the Atlantic, Nellie took just one bag in order to move
 quickly. 
 How is this detail important to paragraph 1?
 Correct answer: It shows one way to help her reach her goal.

 Wrong answer 1: It shows how she is the same as the character in the book.

 Wrong answer 2: It shows a young reporter exploring the world in real life.

 Wrong answer 3: It shows that a young woman taking a trip alone is unusual.
"No one but a man can do this

In [ ]:
f.shape

(1087, 18)

In [6]:

def vectorize_with_pretrained_embeddings(sentences):
  """
  Produces a tensor containing a BERT embedding for each sentence in the dataset or in a
  batch
  Args:
    sentences: List of sentences of length n
  Returns:
    embeddings: A 2D torch array containing embeddings for each of the n sentences (n x d)
                where d = 768
  """


  tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
  pretrained_model = transformers.BertModel.from_pretrained('bert-base-cased', output_hidden_states=False)
  pretrained_model.eval()
  embeddings = []
  for sentence in sentences:
    with_tags = "[CLS] " + sentence + " [SEP]"
    tokenized_sentence = tokenizer.tokenize(with_tags)
    tokenized_sentence = tokenized_sentence[:512]
    # print(tokenized_sentence)
    # print(len(tokenized_sentence))
    indices_from_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    segments_ids = [1] * len(indices_from_tokens)
    tokens_tensor = torch.tensor([indices_from_tokens])
    segments_tensors = torch.tensor([segments_ids])
    # print(indices_from_tokens)
    # print(tokens_tensor)
    # print(segments_tensors)
    with torch.no_grad():
      outputs = pretrained_model(tokens_tensor, segments_tensors)[0] # The output is the
      #last hidden state of the pretrained model of shape 1 x sentence_length x BERT embedding_length
      embeddings.append(torch.mean(outputs, dim = 1))# we average across the embedding length
      #dimension to produce constant sized tensors
  print(embeddings[0].shape)
  embeddings = torch.cat(embeddings, dim = 0)
  print('Shape of embeddings tensor (n x d = 768): ', embeddings.shape)
  return embeddings.cpu().detach().numpy()


In [ ]:

combined_input = ["The boy is running", "The dog has been barking for the whole evening"]
print(combined_input)
embedding = vectorize_with_pretrained_embeddings(combined_input)
print(embedding.shape)

['The boy is running', 'The dog has been barking for the whole evening']


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([2, 768])
(2, 768)


In [ ]:
#Test sentences. To test, run 'python pretrained_model_vectorizer.py' at the command line
#sentences = ["The boy is running", "The dog has been barking for the whole evening"]
#To use with real data, import pretrained_model_vectorizer, then call
#'vectorize_with_pretrained_embeddings' on your list of sentences to embed
#question.embed= vectorize_with_pretrained_embeddings(question)

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1116, 768])


In [ ]:
# quesoptionembed_corr= vectorize_with_pretrained_embeddings(ques_correct_answer)
# quesoptionembed1= vectorize_with_pretrained_embeddings(ques_distractor1)
# quesoptionembed2= vectorize_with_pretrained_embeddings(ques_distractor2)
# quesoptionembed3= vectorize_with_pretrained_embeddings(ques_distractor3)

In [ ]:
# quesembed= vectorize_with_pretrained_embeddings(question)
# corrembed= vectorize_with_pretrained_embeddings(option1)
# optionembed1= vectorize_with_pretrained_embeddings(option2)
# optionembed2= vectorize_with_pretrained_embeddings(option3)
# optionembed3= vectorize_with_pretrained_embeddings(option4)

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])


## All distractors and options with tags

In [ ]:
ques_alldistractors= vectorize_with_pretrained_embeddings(ques_alloptions_tag)

In [ ]:
import numpy as np
from numpy import savetxt
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df= pd.DataFrame(ques_alldistractors)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_alldistractors_embed.csv')


In [7]:
from numpy import dot
from numpy.linalg import norm
corrembedtag= vectorize_with_pretrained_embeddings(ques_correct_answer_tag)
optionembed1tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)
optionembed2tag= vectorize_with_pretrained_embeddings(ques_distractor2_tag)
optionembed3tag= vectorize_with_pretrained_embeddings(ques_distractor3_tag)



# cos_sim_cor_dis1 = dot(corrembedtag, optionembed1tag)/(norm(corrembedtag)*norm(optionembed1tag))
# cos_sim_cor_dis2 = dot(corrembedtag, optionembed2tag)/(norm(corrembedtag)*norm(optionembed2tag))
# cos_sim_cor_dis3 = dot(corrembedtag, optionembed3tag)/(norm(corrembedtag)*norm(optionembed3tag))


# Compute cosine similarity using dot product and norms


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])


NameError: name 'np' is not defined

## Questions and distractors with tags: No passage

In [ ]:
ques_alldistractors_nopass= vectorize_with_pretrained_embeddings(ques_alloptions_nopass)
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df= pd.DataFrame(ques_alldistractors_nopass)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_options_nopass.csv')


## Cosine similarity between question + options

In [ ]:
from numpy import dot
from numpy.linalg import norm

# # Tag + Question + Correct answer
ques_correct_answer_tag= 'Question:' + f['QuestionText'] + '\n Correct answer: ' + f['option_correct_ans']
ques_distractor1_tag=  'Question:' + f['QuestionText'] + '\n Wrong answer 1: ' + f['option_distractor1']
ques_distractor2_tag= 'Question:' + f['QuestionText'] + '\n Wrong answer 2: ' + f['option_distractor2']
ques_distractor3_tag= 'Question:' + f['QuestionText'] + '\n Wrong answer 3: ' + f['option_distractor3']


corrembedtag= vectorize_with_pretrained_embeddings(ques_correct_answer_tag)
optionembed1tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)
optionembed2tag= vectorize_with_pretrained_embeddings(ques_distractor2_tag)
optionembed3tag= vectorize_with_pretrained_embeddings(ques_distractor3_tag)


# Compute cosine similarity using dot product and norms
cosine_similarities1 = np.sum(corrembedtag * optionembed1tag, axis=1) / (norm(corrembedtag, axis=1) * norm(optionembed1tag, axis=1))
cosine_similarities2 = np.sum(corrembedtag * optionembed2tag, axis=1) / (norm(corrembedtag, axis=1) * norm(optionembed2tag, axis=1))
cosine_similarities3 = np.sum(corrembedtag * optionembed3tag, axis=1) / (norm(corrembedtag, axis=1) * norm(optionembed3tag, axis=1))



In [14]:
cosine_similarities1.shape
from scipy import stats
print(stats.describe(cosine_similarities1))
print(stats.describe(cosine_similarities2))
print(stats.describe(cosine_similarities3))

DescribeResult(nobs=1087, minmax=(0.9190467, 0.99854875), mean=0.9737685, variance=0.00015498176584762916, skewness=-0.6916731403302991, kurtosis=0.7278573758042235)
DescribeResult(nobs=1087, minmax=(0.9135256, 0.9984705), mean=0.9738783, variance=0.0001600263460998511, skewness=-0.803287379215649, kurtosis=1.0775616881621728)
DescribeResult(nobs=1087, minmax=(0.91514486, 0.998364), mean=0.9736919, variance=0.0001506499247089488, skewness=-0.6380642766540986, kurtosis=0.6468695025346789)


In [21]:
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df = np.column_stack([cosine_similarities1, cosine_similarities2, cosine_similarities3])
df= pd.DataFrame(df)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']

df.to_csv('question_embed_cos_sim.csv')

/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings


,0,1,2,PassNumUnq,QNoUnq
0,0.981608,0.981462,0.983269,2019-released-items-ela-g4.1,2019-released-items-ela-g4.1.13
1,0.978312,0.979026,0.980008,2019-released-items-ela-g4.1,2019-released-items-ela-g4.1.14
2,0.966038,0.950966,0.957291,2019-released-items-ela-g4.1,2019-released-items-ela-g4.1.15
3,0.984400,0.981751,0.983427,2019-released-items-ela-g4.1,2019-released-items-ela-g4.1.16
4,0.962538,0.962351,0.959325,2019-released-items-ela-g4.1,2019-released-items-ela-g4.1.17


## Cosine similarity between option texts

In [22]:


# # Tag + Question + Correct answer
ques_correct_answer_tag= '\n Correct answer: ' + f['option_correct_ans']
ques_distractor1_tag=  '\n Wrong answer 1: ' + f['option_distractor1']
ques_distractor2_tag= '\n Wrong answer 2: ' + f['option_distractor2']
ques_distractor3_tag= '\n Wrong answer 3: ' + f['option_distractor3']


corrembedtag= vectorize_with_pretrained_embeddings(ques_correct_answer_tag)
optionembed1tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)
optionembed2tag= vectorize_with_pretrained_embeddings(ques_distractor2_tag)
optionembed3tag= vectorize_with_pretrained_embeddings(ques_distractor3_tag)


# Compute cosine similarity using dot product and norms
cosine_similarities1 = np.sum(corrembedtag * optionembed1tag, axis=1) / (norm(corrembedtag, axis=1) * norm(optionembed1tag, axis=1))
cosine_similarities2 = np.sum(corrembedtag * optionembed2tag, axis=1) / (norm(corrembedtag, axis=1) * norm(optionembed2tag, axis=1))
cosine_similarities3 = np.sum(corrembedtag * optionembed3tag, axis=1) / (norm(corrembedtag, axis=1) * norm(optionembed3tag, axis=1))

print(cosine_similarities1.shape)
from scipy import stats
print(stats.describe(cosine_similarities1))
print(stats.describe(cosine_similarities2))
print(stats.describe(cosine_similarities3))

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])
(1087,)
DescribeResult(nobs=1087, minmax=(0.7968028, 0.975649), mean=0.9031167, variance=0.0008279564630705006, skewness=-0.3392121232848891, kurtosis=0.21750817872949657)
DescribeResult(nobs=1087, minmax=(0.78169423, 0.9742325), mean=0.9037173, variance=0.0008617055209067995, skewness=-0.4876761696283755, kurtosis=0.551650368704748)
DescribeResult(nobs=1087, minmax=(0.7785661, 0.97779125), mean=0.9023937, variance=0.0009006838763173855, skewness=-0.5587201965266023, kurtosis=0.7598730574699473)


In [23]:
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df = np.column_stack([cosine_similarities1, cosine_similarities2, cosine_similarities3])
df= pd.DataFrame(df)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']

df.to_csv('options_embed_cos_sim.csv')

/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings


## All Permutations

In [ ]:
nr= f.shape[0]
list_PassNumUnq = []
list_QNoUnq = []
list_permute_id = []
list_text = []
list_score = []
for i in range(nr):
  #print(f["option_correct_ans"][i])
  answer = [
            "Correct answer: " + f["option_correct_ans"][i],
            "Wrong answer 1: " + f["option_distractor1"][i],
            "Wrong answer 2: " + f["option_distractor2"][i],
            "Wrong answer 3: " + f["option_distractor3"][i],
        ]
  answers = list(itertools.permutations(answer))
  for permute_id, answer in enumerate(answers):
    text = (f['QuestionText'][i] + "\n" +
                answer[0] + "\n" +
                answer[1] + "\n" +
                answer[2] + "\n" +
                answer[3] + "\n" +
                f['Passage'][i]
            )
    list_PassNumUnq.append(f['PassNumUnq'][i])
    list_QNoUnq.append(f['QNoUnq'][i])
    list_permute_id.append(permute_id)
    list_text.append(text)
    list_score.append(f['pVal'][i])

ques_alloptions_tag= vectorize_with_pretrained_embeddings(list_text)

%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df= pd.DataFrame(ques_alloptions_tag)
df['PassNumUnq'] = list_PassNumUnq
df['QNoUnq'] = list_QNoUnq
df['permutation']= list_permute_id

df.to_csv('question_embed_permutations.csv')




In [ ]:

%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df= pd.DataFrame(ques_alldistractors)
df['PassNumUnq'] = list_PassNumUnq
df['QNoUnq'] = list_QNoUnq
df['permutation']= list_permute_id

df.to_csv('question_embed_permutations.csv')


/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings


torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1087, 768])


In [ ]:


# %cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
# df= pd.DataFrame(quesoptionembed_corr)
# df['PassNumUnq'] = f['PassNumUnq']
# df['QNoUnq'] = f['QNoUnq']
# df.to_csv('question_embed_correct.csv')

# df= pd.DataFrame(quesoptionembed1)
# df['PassNumUnq'] = f['PassNumUnq']
# df['QNoUnq'] = f['QNoUnq']
# df.to_csv('question_embed_dis1.csv')

# df= pd.DataFrame(quesoptionembed2)
# df['PassNumUnq'] = f['PassNumUnq']
# df['QNoUnq'] = f['QNoUnq']
# df.to_csv('question_embed_dis2.csv')

# df= pd.DataFrame(quesoptionembed3)
# df['PassNumUnq'] = f['PassNumUnq']
# df['QNoUnq'] = f['QNoUnq']
# df.to_csv('question_embed_dis3.csv')



#df= pd.DataFrame(quesembed)
#df.to_csv('question_embed.csv')

#df= pd.DataFrame(corrembed)
#df.to_csv('option_embed_correct.csv')

#df= pd.DataFrame(optionembed1)
#df.to_csv('optionembed1.csv')

#df= pd.DataFrame(optionembed2)
#f.to_csv('optionembed2.csv')

#df= pd.DataFrame(optionembed3)
#df.to_csv('optionembed3.csv')

#np.savetxt('/Users/radhika/Downloads/question_embed.csv', question.embed, delimiter=',')
#np.save(question.embed,)


/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings


In [ ]:
corrembedtag= vectorize_with_pretrained_embeddings(ques_correct_answer_tag)
optionembed1tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)
optionembed2tag= vectorize_with_pretrained_embeddings(ques_distractor2_tag)
optionembed3tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)

KeyboardInterrupt: 

In [ ]:
# import numpy as np
# from numpy import savetxt
# %cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'

# df= pd.DataFrame(corrembedtag)
# df.to_csv('option_embed_correct_tag.csv')

# df= pd.DataFrame(optionembed1tag)
# df.to_csv('optionembed1_tag.csv')

# df= pd.DataFrame(optionembed2tag)
# df.to_csv('optionembed2_tag.csv')

# df= pd.DataFrame(optionembed3tag)
# df.to_csv('optionembed3_tag.csv')

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.330693,0.038969,0.069046,0.141347,0.562531,-0.034794,-0.045604,0.071342,-0.053162,-0.217772,...,0.317475,0.156087,-0.123962,-0.269952,0.094822,0.145907,-0.207998,0.324924,0.330233,-0.009789
1,0.150200,0.274270,-0.039747,0.272531,0.666258,-0.126865,0.024354,-0.013578,-0.016012,-0.114078,...,0.208553,0.335136,-0.002739,-0.248087,0.044126,0.209430,-0.082822,0.187699,0.172975,0.142289
2,0.316193,-0.114018,-0.007403,0.150899,0.482411,-0.054715,0.255445,0.073818,-0.062459,-0.065642,...,0.376271,0.203655,-0.218886,-0.109045,0.187939,0.178523,-0.090680,0.405097,0.209188,-0.032180
3,0.451735,-0.035361,0.060584,0.046701,0.494822,0.024858,0.058867,0.131332,-0.004860,-0.290856,...,0.245316,0.166824,-0.153981,-0.259816,-0.038695,0.186564,-0.191631,0.309455,0.253131,0.001289
4,0.375125,-0.001411,0.016060,-0.089303,0.024385,0.141236,0.317772,0.116051,-0.206724,-0.230045,...,0.256846,0.177193,-0.099984,-0.219399,0.038975,0.209707,-0.124114,0.087240,0.294826,0.216154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,0.374191,-0.092379,0.249997,0.309695,0.122369,-0.153756,0.123190,0.003419,0.053187,0.195876,...,-0.048363,0.057822,-0.082947,-0.117225,0.034189,-0.216345,-0.063492,0.308002,0.208320,-0.122181
1112,0.197396,-0.015915,0.096769,0.203303,0.251318,-0.227677,0.172645,-0.086674,-0.062594,-0.002558,...,0.036880,0.011768,0.054642,0.074262,0.104311,-0.230042,-0.028243,0.311628,-0.011404,-0.008374
1113,0.297722,-0.003906,0.007676,0.123774,0.052911,-0.238047,0.119631,0.062502,0.060466,0.146408,...,0.037549,0.198642,-0.075447,-0.001407,0.087634,-0.141926,-0.089317,0.286568,0.183502,-0.043947
1114,0.408570,0.140626,0.007848,0.179747,0.132918,-0.180473,0.132025,0.076565,-0.043467,0.182478,...,0.205288,0.061859,-0.020326,-0.103721,0.089978,-0.167842,-0.041928,0.132927,0.166269,-0.081589


In [ ]:
df

(1116, 768)

In [ ]:
question.embed

array([[ 0.33069327,  0.0389686 ,  0.06904568, ...,  0.3249241 ,
         0.33023322, -0.00978899],
       [ 0.15020001,  0.27427033, -0.03974681, ...,  0.18769884,
         0.17297511,  0.14228924],
       [ 0.31619295, -0.11401785, -0.00740315, ...,  0.40509737,
         0.20918822, -0.03217997],
       ...,
       [ 0.29772183, -0.00390649,  0.00767591, ...,  0.28656772,
         0.18350151, -0.04394712],
       [ 0.4085702 ,  0.14062564,  0.00784751, ...,  0.13292709,
         0.1662686 , -0.0815894 ],
       [ 0.359874  ,  0.0653155 , -0.03084283, ...,  0.26369065,
         0.03838476, -0.06845871]], dtype=float32)